<a href="https://www.kaggle.com/code/ismetsemedov/damaged-parcels?scriptVersionId=216593005" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxeswithdamagesanddents56.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxeswithdamagesanddents30.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/damagedfoodpackagingbox59.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged234.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged245.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/damagedfoodpackagingbox244.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged18.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged109.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packag

In [2]:
# Install required packages
!pip install ultralytics roboflow scikit-learn opencv-python torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [3]:
# Import required libraries
from ultralytics import YOLO  # Import YOLO model from ultralytics
import os  # For file and directory operations
import torch  # PyTorch for deep learning
from sklearn.model_selection import train_test_split  # For splitting dataset
import shutil  # For file copying operations

def create_yolo_dataset():
    """
    Creates the required directory structure for YOLO training.
    Directory structure:
    /dataset
        /images
            /train  - Training images
            /val    - Validation images
        /labels
            /train  - Training labels
            /val    - Validation labels
    """
    base_dir = '/kaggle/working'  # Kaggle's working directory
    # Create directories with exist_ok=True to avoid errors if directories exist
    os.makedirs(f'{base_dir}/dataset/images/train', exist_ok=True)
    os.makedirs(f'{base_dir}/dataset/images/val', exist_ok=True)
    os.makedirs(f'{base_dir}/dataset/labels/train', exist_ok=True)
    os.makedirs(f'{base_dir}/dataset/labels/val', exist_ok=True)

def prepare_data():
    """
    Prepares the dataset by:
    1. Loading image paths for damaged and intact parcels
    2. Splitting data into train and validation sets
    3. Copying images to appropriate directories
    4. Creating corresponding label files
    """
    # Define paths for damaged and intact parcel images
    damaged_dir = '/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged'
    intact_dir = '/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/intact'
    
    # Get list of image files (jpg and jpeg)
    damaged_files = [f for f in os.listdir(damaged_dir) if f.endswith(('.jpg', '.jpeg'))]
    intact_files = [f for f in os.listdir(intact_dir) if f.endswith(('.jpg', '.jpeg'))]
    
    # Split data: 80% training, 20% validation
    damaged_train, damaged_val = train_test_split(damaged_files, test_size=0.2, random_state=42)
    intact_train, intact_val = train_test_split(intact_files, test_size=0.2, random_state=42)
    
    base_dir = '/kaggle/working'
    
    # Process damaged training images
    for img_file in damaged_train:
        # Copy image to training directory
        shutil.copy(os.path.join(damaged_dir, img_file), 
                   os.path.join(f'{base_dir}/dataset/images/train', img_file))
        create_label(img_file, 'train', 0)  # 0 = damaged class
    
    # Process intact training images
    for img_file in intact_train:
        # Copy image to training directory
        shutil.copy(os.path.join(intact_dir, img_file),
                   os.path.join(f'{base_dir}/dataset/images/train', img_file))
        create_label(img_file, 'train', 1)  # 1 = intact class
    
    # Process damaged validation images
    for img_file in damaged_val:
        # Copy image to validation directory
        shutil.copy(os.path.join(damaged_dir, img_file),
                   os.path.join(f'{base_dir}/dataset/images/val', img_file))
        create_label(img_file, 'val', 0)  # 0 = damaged class
    
    # Process intact validation images
    for img_file in intact_val:
        # Copy image to validation directory
        shutil.copy(os.path.join(intact_dir, img_file),
                   os.path.join(f'{base_dir}/dataset/images/val', img_file))
        create_label(img_file, 'val', 1)  # 1 = intact class

def create_label(img_file, split, class_id):
    """
    Creates YOLO format label file for an image.
    Args:
        img_file (str): Image filename
        split (str): 'train' or 'val'
        class_id (int): Class ID (0 for damaged, 1 for intact)
    
    YOLO format: <class_id> <x_center> <y_center> <width> <height>
    All values are normalized to [0, 1]
    """
    base_dir = '/kaggle/working'
    # Create label file path by replacing image extension with .txt
    label_path = f'{base_dir}/dataset/labels/{split}/{os.path.splitext(img_file)[0]}.txt'
    # Write label in YOLO format
    with open(label_path, 'w') as f:
        # Format: class_id x_center y_center width height
        # Using full image bounds (0.5, 0.5, 1.0, 1.0) since we're doing classification
        f.write(f'{class_id} 0.5 0.5 1.0 1.0\n')

def create_yaml():
    """
    Creates YAML configuration file for YOLO training.
    Contains:
    - Paths to train and validation images
    - Number of classes
    - Class names
    """
    yaml_content = """
train: /kaggle/working/dataset/images/train
val: /kaggle/working/dataset/images/val
nc: 2
names: ['damaged', 'intact']
    """
    
    # Write YAML configuration file
    with open('/kaggle/working/dataset.yaml', 'w') as f:
        f.write(yaml_content)

def train_model():
    """
    Initializes and trains the YOLO model.
    Returns:
        model: Trained YOLO model
    
    Training parameters:
    - epochs: 50 training epochs
    - imgsz: 640x640 input image size
    - batch: 16 images per batch
    - workers: 4 data loading workers
    """
    # Load pretrained YOLOv8n (nano) model
    model = YOLO('yolov8n.pt')
    
    # Train the model with specified parameters
    results = model.train(
        data='/kaggle/working/dataset.yaml',  # Dataset configuration file
        epochs=50,                            # Number of epochs
        imgsz=640,                           # Input image size
        batch=16,                            # Batch size
        workers=4,                           # Number of data loading workers
        device='cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
    )
    return model

# Main execution block
if __name__ == "__main__":
    create_yolo_dataset()    # Create directory structure
    prepare_data()           # Prepare and split dataset
    create_yaml()           # Create configuration file
    model = train_model()   # Train the model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 86.3MB/s]


Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 18.0MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50      2.36G    0.06684      0.235     0.8681          1        640: 100%|██████████| 28/28 [00:04<00:00,  5.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.99it/s]

                   all        109        109      0.854      0.886      0.902      0.894



50 epochs completed in 0.090 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.13it/s]


                   all        109        109        0.9      0.858      0.936       0.93
               damaged         51         51      0.864      0.961      0.942      0.928
                intact         58         58      0.936      0.756      0.931      0.931
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/train
